# Final Project

## San Francisco Crime Classification

### Project Problem and Hypothesis

##### The following is an excerpt from Kaggle San Francisco Crime Classification: 

##### With rising wealth inequality, housing shortages, and a proliferation of expensive digital toys riding BART to work, there is no scarcity of crime in the city by the bay

##### My overall summary:
By using this dataset, predict the category of crime that will occur in the city on the test set. To keep this project manageable, I've reduced to the number of categories I'm allowed to predict: predict whether the crime will fall under "Theft", "Assault", "Drug/Narcotic," or "other". An example of how this prediction  could help in the future is to allocate enough law enforcement at the right place and right time to prevent the three classifications. Through some feataure engineering, I'll keep the columns, "X", "Y", "Year", "Month", "Day","Hour", and "Minute", and "DayInNumber". Finally, I will use random forests as suggested by Ivan as the variables are likely to not be linear. 

### Datasets

Year - Year.

Month - Month

Day - Day of the month

Hour - Hour of the day. For example, 20 will be 8 pm.

Minute - Minute of the day. For example, 30 will the minute hand on a clock pointed at 6.

X - Longitude

Y - Latitude

DayInNumber - The day of the week. 1 would be Sunday and 7 would be Saturday.

All text columns were dropped because the RandomForestClassifier() function didn't like taking strings arguments.

### Domain knowledge

Thankfully, if I google "San Francisco Crime Classification github," there is a lot of work that has already been done on this, so I was leverage other ideas if I get stuck.
    
<b>Benchmark</b>: 

After doing some google searches, it looks like the best model for this classifier is from a decision tree, which we haven't covered yet in class. I read someone online that someone got a rank of 1149 with a score of 2.64. I'm not sure what this means, I'm I am adding this comment as part of the document writeup.

### Project Concerns

I'm not sure where to start. I plan on working with the same model my group had used for the class assignment, titantic, and work from there to see what can be done. I'm also reading from other folks who used predictive models that we haven't covered yet in class. Some of these include Naïve Bayes and decision trees. While there are other models out there that were used for this dataset, I'd like to stick with materials we learned in class as much as possible even if that would lead to a weaker model. I need help determining what would be the best route. 

I don't what to do with X and Y variables, although I think they're important to keep.

<b>Assumptions</b>: 

The only high level assumption I can think of for this is that the data entered is as accurate as possible. 

<b>Data I wish I had</b>:

It would be great to figure out how to create heat maps for this. I believe this can be done with the available coordinates, but I'm not sure how to do it.

<b>Risks</b>:

The only risk I can think of is whether or not the actual data entry is accurate. I think the risk is low.



### Outcomes

I'm hoping to train a training set, then test the model against the test set. My goal to get a good percentage of them predicted correctly. Given that I'm new to data science, I will try to keep my model as simple as possible using material from class. For me, success would be to come up with predictions utilizing class material. My best case sceanrio for success is a high score for predictions using class material.


In [196]:
import os

import math

import numpy as np
import pandas as pd
from datetime import datetime as dt
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 12)

from sklearn import linear_model, cross_validation, tree, ensemble, metrics, model_selection, externals
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

#take out x and y because it's probably not useful
#probably use logisitic regression.
#will go into decision trees so this might work.
#You can plot x and y.

In [197]:
df = pd.read_csv("train.csv")

In [198]:
df

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607


### Exploratory analysis

In [199]:
df.Address.value_counts() #count the number of times an address occurs.
# Most crime occurs on Bryant ST

800 Block of BRYANT ST      26533
800 Block of MARKET ST       6581
2000 Block of MISSION ST     5097
1000 Block of POTRERO AV     4063
900 Block of MARKET ST       3251
                            ...  
2600 Block of TARAVAL ST        1
0 Block of PALOALTO AV          1
ALEMANY BL / RAMSELL ST         1
2800 Block of MARKET ST         1
KRAMER PL / GREENWICH ST        1
Name: Address, dtype: int64

In [200]:
df.columns # see all features

Index([u'Dates', u'Category', u'Descript', u'DayOfWeek', u'PdDistrict',
       u'Resolution', u'Address', u'X', u'Y'],
      dtype='object')

In [201]:
df.Resolution.unique() # see all unique resolutions 

array(['ARREST, BOOKED', 'NONE', 'ARREST, CITED', 'PSYCHOPATHIC CASE',
       'JUVENILE BOOKED', 'UNFOUNDED', 'EXCEPTIONAL CLEARANCE', 'LOCATED',
       'CLEARED-CONTACT JUVENILE FOR MORE INFO', 'NOT PROSECUTED',
       'JUVENILE DIVERTED', 'COMPLAINANT REFUSES TO PROSECUTE',
       'JUVENILE ADMONISHED', 'JUVENILE CITED',
       'DISTRICT ATTORNEY REFUSES TO PROSECUTE',
       'PROSECUTED BY OUTSIDE AGENCY', 'PROSECUTED FOR LESSER OFFENSE'], dtype=object)

In [202]:
df.Category.unique() # see all unique categories of crimes 

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING', 'SEX OFFENSES NON FORCIBLE',
       'EXTORTION', 'GAMBLING', 'BAD CHECKS', 'TREA', 'RECOVERED VEHICLE',
       'PORNOGRAPHY/OBSCENE MAT'], dtype=object)

In [203]:
df.Category.value_counts() # see 5 most popular crimes

LARCENY/THEFT                174900
OTHER OFFENSES               126182
NON-CRIMINAL                  92304
ASSAULT                       76876
DRUG/NARCOTIC                 53971
                              ...  
EXTORTION                       256
SEX OFFENSES NON FORCIBLE       148
GAMBLING                        146
PORNOGRAPHY/OBSCENE MAT          22
TREA                              6
Name: Category, dtype: int64

In [204]:
df = df.drop(['Descript'], axis = 1) # drop Descript column

In [205]:
df.PdDistrict.unique().tolist() # show all unique districts

['NORTHERN',
 'PARK',
 'INGLESIDE',
 'BAYVIEW',
 'RICHMOND',
 'CENTRAL',
 'TARAVAL',
 'TENDERLOIN',
 'MISSION',
 'SOUTHERN']

In [206]:
df.head()

,Dates,Category,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [207]:
#testing out daytime package. Run the time right now.
now = dt.now().strftime("%Y-%m-%d_%H:%M:%S.%f")

print dt.now()

now

2017-06-25 13:23:51.083738


'2017-06-25_13:23:51.083664'

In [208]:
df['Dates'] = pd.to_datetime(df['Dates']) # Convert original Date column to correct type to use striftime below

In [209]:
#df['Dates2'] = df['Dates'].map(lambda x: x.strftime('%Y-%m-%d')) this code works
df['Year'] = df['Dates'].map(lambda x: x.strftime('%Y'))

In [210]:
df['Month'] = df['Dates'].map(lambda x: x.strftime('%m'))
df['Day'] = df['Dates'].map(lambda x: x.strftime('%d'))
df['Hour'] = df['Dates'].map(lambda x: x.strftime('%H'))
df['Minute'] = df['Dates'].map(lambda x: x.strftime('%M'))


In [211]:
df.Category.unique().tolist() # show all unique Categories

['WARRANTS',
 'OTHER OFFENSES',
 'LARCENY/THEFT',
 'VEHICLE THEFT',
 'VANDALISM',
 'NON-CRIMINAL',
 'ROBBERY',
 'ASSAULT',
 'WEAPON LAWS',
 'BURGLARY',
 'SUSPICIOUS OCC',
 'DRUNKENNESS',
 'FORGERY/COUNTERFEITING',
 'DRUG/NARCOTIC',
 'STOLEN PROPERTY',
 'SECONDARY CODES',
 'TRESPASS',
 'MISSING PERSON',
 'FRAUD',
 'KIDNAPPING',
 'RUNAWAY',
 'DRIVING UNDER THE INFLUENCE',
 'SEX OFFENSES FORCIBLE',
 'PROSTITUTION',
 'DISORDERLY CONDUCT',
 'ARSON',
 'FAMILY OFFENSES',
 'LIQUOR LAWS',
 'BRIBERY',
 'EMBEZZLEMENT',
 'SUICIDE',
 'LOITERING',
 'SEX OFFENSES NON FORCIBLE',
 'EXTORTION',
 'GAMBLING',
 'BAD CHECKS',
 'TREA',
 'RECOVERED VEHICLE',
 'PORNOGRAPHY/OBSCENE MAT']

In [212]:
#recategorize boarder categories

#df['HighLevelCategory'] = df.Category.apply(lambda NameOfCategory:"Theft" if NameOfCategory =="LARCENY/THEFT" else "Other")

#df['HighLevelCategory'] = df.Category.apply(lambda NameOfCategory:"Theft" 
                                            #if NameOfCategory =="LARCENY/THEFT" or NameOfCategory == "VEHICLE THEFT" or NameOfCategory == "BURGLARY" or NameOfCategory == "STOLEN PROPERTY" or NameOfCategory == "Fraud" 
                                            #else "Other")

#df['HighLevelCategory'] = df.Category.apply(lambda NameOfCategory:"Theft" 
                                           # if NameOfCategory =="LARCENY/THEFT" or NameOfCategory == "VEHICLE THEFT" or NameOfCategory == "BURGLARY" or NameOfCategory == "STOLEN PROPERTY" or NameOfCategory == "Fraud" 
                                            #else ("ASSAULT" if NameOfCategory == "ASSAULT" else ("NARCOTIC" if NameOfCategory == "NARCOTIC" else "Other")))
        
def category_to_high_level(Category):
    if Category in ["LARCENY/THEFT", "VEHICLE THEFT", "BURGLARY", "STOLEN PROPERTY", "FRAUD"]:
       return "Theft"
    elif Category in ["ASSAULT"]:
       return "Assault"
    elif Category in ["DRUG/NARCOTIC"]:
       return "Drug/Narcotic"               
    else:
       return "Other"
                    
df['HighLevelCategory'] = df.Category.apply(category_to_high_level)

def DayOfWeek_to_numbers(week):
    if week in ["Sunday"]:
       return "1"
    elif week in ["Monday"]:
       return "2"
    elif week in ["Tuesday"]:
       return "3"
    elif week in ["Wednesday"]:
       return "4"
    elif week in ["Thursday"]:
       return "5"
    elif week in ["Friday"]:
       return "6"             
    else:
       return "7"
                    
df['DayInNumber'] = df.DayOfWeek.apply(DayOfWeek_to_numbers)



In [213]:
df.HighLevelCategory.unique().tolist() #check

['Other', 'Theft', 'Assault', 'Drug/Narcotic']

In [214]:
df.DayInNumber.unique().tolist() #check

['4', '3', '2', '1', '7', '6', '5']

In [215]:
df.HighLevelCategory.value_counts()

Other            460547
Theft            286655
Assault           76876
Drug/Narcotic     53971
Name: HighLevelCategory, dtype: int64

In [216]:
X = df[df.columns.values]
X.drop(['HighLevelCategory','Address','Category','DayOfWeek','PdDistrict','Resolution','Dates'], axis = 1, inplace = True) #dropped address because of model error RandomForestClassifier

y = df.HighLevelCategory

X

,X,Y,Year,Month,Day,Hour,Minute,DayInNumber
0,-122.425892,37.774599,2015,05,13,23,53,4
1,-122.425892,37.774599,2015,05,13,23,53,4
2,-122.424363,37.800414,2015,05,13,23,33,4
3,-122.426995,37.800873,2015,05,13,23,30,4
4,-122.438738,37.771541,2015,05,13,23,30,4
...,...,...,...,...,...,...,...,...
878044,-122.459033,37.714056,2003,01,06,00,15,2
878045,-122.447364,37.731948,2003,01,06,00,01,2
878046,-122.403390,37.780266,2003,01,06,00,01,2
878047,-122.390531,37.780607,2003,01,06,00,01,2


In [217]:
y

0         Other
1         Other
2         Other
3         Theft
4         Theft
          ...  
878044    Other
878045    Theft
878046    Theft
878047    Other
878048    Other
Name: HighLevelCategory, dtype: object

### Split dataset into a training and testing sets (60%, 40%)

In [218]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(X, y, train_size = .6, random_state = 0)


In [219]:
model = ensemble.RandomForestClassifier()

params = {"n_estimators": [1, 10, 100],
          "min_samples_leaf": [1, 5, 10],
          "oob_score": [True, False]}

gs = GridSearchCV(model, params, cv = 3)

gs.fit (train_X, train_y)

#code above figures out best combinations


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 10, 100], 'oob_score': [True, False], 'min_samples_leaf': [1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [220]:
gs.best_params_ # this displays the best combinations

{'min_samples_leaf': 5, 'n_estimators': 100, 'oob_score': False}

In [221]:
best_forest = gs.best_estimator_ #this IS the best model. It's shorter version of Ivan's example.

best_forest.fit(train_X, train_y) # reports the default parameters.

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [222]:
train_y_hat1 = model_selection.cross_val_predict(best_forest, train_X, train_y, cv = 5) #remember that cross validation already does a .fit


In [223]:
pd.crosstab(train_y_hat1, train_y, rownames = ['Hypothesized Class'], colnames = ['True Class'])

True Class,Assault,Drug/Narcotic,Other,Theft
Hypothesized Class,,,,
Assault,153,4,260,12
Drug/Narcotic,148,3205,2946,128
Other,38685,27958,222475,82345
Theft,7011,1216,50307,89976


In [224]:
train_y_hat2 = best_forest.predict(train_X)

In [225]:
pd.crosstab(train_y_hat2, train_y, rownames = ['Hypothesized Class'], colnames = ['True Class'])

True Class,Assault,Drug/Narcotic,Other,Theft
Hypothesized Class,,,,
Assault,1881,9,62,19
Drug/Narcotic,151,9404,1002,136
Other,37306,21829,259630,37170
Theft,6659,1141,15294,135136


In [226]:
best_forest.oob_score_
#how well are you doing on training data you're not using on your bootstrap.

AttributeError: 'RandomForestClassifier' object has no attribute 'oob_score_'

### 5-fold cross-validation error on your training set

In [227]:
model_selection.cross_val_score(best_forest, train_X, train_y, cv = 5).mean()

0.59953988747778797

### Training Scores

In [228]:
metrics.accuracy_score(train_y, train_y_hat1)

0.59945257379529227

In [229]:
metrics.accuracy_score(train_y, train_y_hat2)

0.7707453462129078

In [230]:
best_forest.score(train_X, train_y)

0.7707453462129078

### Test 

In [231]:
best_forest.score(test_X, test_y)

0.60101645692158756

In [232]:
test_y_hat1 = model_selection.cross_val_predict(best_forest, test_X, test_y, cv = 5) #remember that cross validation already does a .fit


In [233]:
pd.crosstab(test_y_hat1, test_y, rownames = ['Hypothesized Class'], colnames = ['True Class'])

True Class,Assault,Drug/Narcotic,Other,Theft
Hypothesized Class,,,,
Assault,42,2,76,6
Drug/Narcotic,83,1764,1565,80
Other,26010,18935,149843,57148
Theft,4744,887,33075,56960


In [234]:
test_y_hat2 = best_forest.predict(test_X)

In [235]:
pd.crosstab(test_y_hat2, test_y, rownames = ['Hypothesized Class'], colnames = ['True Class'])

True Class,Assault,Drug/Narcotic,Other,Theft
Hypothesized Class,,,,
Assault,141,6,237,13
Drug/Narcotic,92,2392,2236,81
Other,26063,18344,148282,53826
Theft,4583,846,33804,60274


In [236]:
metrics.accuracy_score(test_y, test_y_hat1)

0.59395535561756163

In [237]:
metrics.accuracy_score(test_y, test_y_hat2)

0.60101645692158756